In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch

text = "Он думает только о " + tokenizer.mask_token + ", и совсем не думает о слухах на " + tokenizer.mask_token + "."

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-uncased', return_dict=True)

input_data = tokenizer.encode_plus(text, return_tensors="pt")
mask_indices = torch.where(input_data["input_ids"][0] == tokenizer.mask_token_id)

# Получение предсказаний модели
output = model(**input_data)
logits = output.logits

# softmax для получения вероятностей
probabilities = F.softmax(logits, dim=-1)



print(f"{'№':<4} {'Word':<20} {'Weight':<10} {'Match':<10}")
print("-" * 50)

i = 1
for mask_idx in mask_indices[0]:
    mask_probabilities = probabilities[0, mask_idx, :]
    top_predictions = torch.topk(mask_probabilities, 10)

    for token_id, score in zip(top_predictions.indices, top_predictions.values):
        predicted_word = tokenizer.decode([token_id])
        is_match = "[Match]" if predicted_word in {"себе", "работе"} else ""
        
        print(f"{i:<4} {predicted_word:<20} {score.item():<10.4f} {is_match:<10}")
        
        i += 1
        if i == 10: break
    if i == 10: break 

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


№    Word                 Weight     Match     
--------------------------------------------------
1    себе                 0.2319     [Match]   
2    нем                  0.1303               
3    том                  0.0641               
4    любви                0.0547               
5    работе               0.0470     [Match]   
6    жизни                0.0384               
7    неи                  0.0355               
8    деле                 0.0265               
9    них                  0.0235               
